In [ ]:
from configparser import ConfigParser

config = ConfigParser()
config.read('config.ini')
config['augmentations']['pad_trunc_noise_multiplier']


In [ ]:
import torch
import ml.machineLearning as machineLearning

model = machineLearning.selectModel()

In [ ]:
from torchsummary import summary
summary(model, (1,257, 63))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

file =  './data/JQ_rec.txt'

label_list = [0]
time_list = [0]

with open(file) as f:
    for lbl in f.read().splitlines():
        lbl = lbl.split('\t')

        startTime = int(float(lbl[0]))
        endTime = int(float(lbl[1]))
        num_speakers = int(lbl[2])

        if startTime != time_list[-1]:
            time_list.append(startTime)
            label_list.append(1)
        time_list.append(endTime)
        label_list.append(int(num_speakers))
print(label_list)
print(time_list)
plt.step(time_list, label_list)

In [9]:
import torch
from loader.AudioDataset import specMask, collate_batch, createDataset
from torch.utils.data import Dataset
import loader.utils as utils
import matplotlib.pyplot as plt

linux = '/media/jianquan/Data/Processed Audio/'
windows = './data/WAVE/'

audio_paths, _ = utils.getAudioPaths(linux)

audio_train_dataset = createDataset(audio_paths, transformParams = utils.getTransforms(False), outputAudio = True)

test_dataloader = torch.utils.data.DataLoader(
    audio_train_dataset,
    batch_size=1,
    num_workers=0,
    shuffle=True,
    collate_fn = collate_batch
)

batch = next(iter(test_dataloader))
batch

print(len(batch[0]))

import IPython.display as ipd
ipd.Audio(batch[0][3], rate = 8000)
# for b in batch[0]:
#     # print(b.max(),b.median(),b.mean(), b.min())
#     # print(b).
    
#     plt.plot(b[0])
#     plt.show()

300


In [ ]:
import librosa
import librosa.display
import torchaudio
import numpy as np

wav, sr = librosa.load('./data/JQ_rec.wav')

spec = np.abs(librosa.stft(wav[32000:64000], n_fft = 128))

spec = spec/(spec + np.median(spec))

import matplotlib.pyplot as plt

plt.imshow(spec)

In [ ]:
import matplotlib.pyplot as plt
import torch, torchaudio

a = torchaudio.load('/media/jianquan/Data/Original Audio/Singapore Speech Corpus/[P] Part 3 Same BoundaryMic/3003.wav')[0]
b = torchaudio.load('/media/jianquan/Data/Processed Audio/SPEECH/4 Diff Room/sur_2010_6020_phnd_cs-mly.wav')[0]
a

In [ ]:
a = a[0][0:32000]
b = b[0][0:32000]
a.shape

In [10]:
import torch

for i in range(1000):
    torch.tensor([0,1,2]).repeat(100)